# 코로나 감염 시·도발생 현황 API

- 보건복지부
- https://www.data.go.kr/iim/api/selectAPIAcountView.do

- 코로나19감염증으로 인한 시.도별 신규확진자,신규사망자,격리중인환자수,격리해제환자수등에 대한 현황자료

- 03.보건복지부_OpenAPI활용가이드_코로나바이러스감염증_시도발생_현황 조회 서비스_v0.3 (2).docx 참고 하여 작성


- 날짜 / 시도명 / 확진자 수 / 지역발생 수

- resultCode	결과코드
- resultMsg	결과메세지
- numOfRows	한 페이지 결과 수
- pageNo	페이지 수
- totalCount	전체 결과 수
- SEQ	게시글번호(국내 시도별 발생현황 고유값)
- CREATE_DT 등록일시분초
- DEATH_CNT 사망자 수
- **GUBUN 시도명(한글)**
- GUBUN_CN 시도명(중국어)
- GUBUN_EN 시도명(영어)
- INC_DEC 전일대비 증감수
- ISOL_CLEAR_CNT 격리 해제 수
- QUR_RATE 10만명당 발생률
- **STD_DAY 기준일시**
- UPDATE_DT 수정일시분초
- **DEF_CNT 확진자 수** : 누적 확진자 수
- ISOL_ING_CNT 격리중 환자수
- OVER_FLOW_CNT 해외유입 수
- **LOCAL_OCC_CNT 지역발생 수** : 일별 확진자 수


## 데이터 불러오기

In [1]:
import urllib.request
import json
import pandas as pd
from datetime import datetime,timedelta
import requests
import xmltodict

## 코로나19감염증으로 인한 일별 확진자, 완치자, 치료중인환자, 사망자등에 대한 현황자료
url_base = 'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19SidoInfStateJson'
url_serviceKey = 'zDi7%2BxdkbZUvy%2Bw2jREuuky26qthb6FliCJd4uJTY8qOo02u2VFch2k2zvE0GKtsF8nLCKxd7WqoNvU%2BuvCaUQ%3D%3D'
url_numOfRows = '1000'
url_start_date = '20200120'
url_end_date = datetime.today().strftime('%Y%m%d') # 현재시간 기준으로 데이터 불러오기

# Call Back URL + 인증키 +  페이지 번호(= 1) + 한 페이지 결과 수 + 데이터 생성일 시작 범위 + 데이터 생성일 종료 범위
url = url_base + '?serviceKey=' + url_serviceKey + '&pageNo=1' + '&num0fRows=' + url_numOfRows + '&startCreateDt=' + url_start_date + '&endCreateDt=' + url_end_date
url

'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19SidoInfStateJson?serviceKey=zDi7%2BxdkbZUvy%2Bw2jREuuky26qthb6FliCJd4uJTY8qOo02u2VFch2k2zvE0GKtsF8nLCKxd7WqoNvU%2BuvCaUQ%3D%3D&pageNo=1&num0fRows=1000&startCreateDt=20200120&endCreateDt=20211201'

In [2]:
req = requests.get(url, allow_redirects=False).content

xmlObject = xmltodict.parse(req)

In [3]:
dict_data = xmlObject['response']['body']['items']['item']

df = pd.DataFrame(dict_data)

In [4]:
pd.isna(df).sum()

createDt        0
deathCnt        0
defCnt          0
gubun           0
gubunCn         0
gubunEn         0
incDec          0
isolClearCnt    0
isolIngCnt      0
localOccCnt     0
overFlowCnt     0
qurRate         0
seq             0
stdDay          0
updateDt        0
dtype: int64

## 필요한 열만 남겨두기 & 열 이름 변경 

In [5]:
df_conf = df.copy()

In [6]:
# 날짜, 시도명(한글), 확진자 수, 지역발생 수
# stdDay, gubun, defCnt, localOccCnt
df_conf = df_conf[['stdDay','gubun','defCnt','localOccCnt']]

df_conf.columns = ['날짜','시도명','확진자 수','지역발생 수']
df_conf

,날짜,시도명,확진자 수,지역발생 수
0,2021년 12월 01일 00시,검역,6469,0
1,2021년 12월 01일 00시,제주,3731,27
2,2021년 12월 01일 00시,경남,15142,126
3,2021년 12월 01일 00시,경북,11365,97
4,2021년 12월 01일 00시,전남,4459,40
...,...,...,...,...
12933,2020년 01월 20일 00시,전남,0,0
12934,2020년 01월 20일 00시,경북,0,0
12935,2020년 01월 20일 00시,경남,0,0
12936,2020년 01월 20일 00시,제주,0,0


## 시도명 으로 묶기

In [7]:
# df_conf = df_conf[''].groupby(df_conf[''])
grouped = df_conf.groupby(['시도명', '날짜'], as_index=False).sum()
grouped

,시도명,날짜,확진자 수,지역발생 수
0,강원,2020년 01월 20일 00시,0,0
1,강원,2020년 01월 21일 00시,0,0
2,강원,2020년 01월 22일 00시,0,0
3,강원,2020년 01월 23일 00시,0,0
4,강원,2020년 01월 24일 00시,0,0
...,...,...,...,...
12933,합계,2021년 11월 27일 00시,436968,4045
12934,합계,2021년 11월 28일 00시,440896,3893
12935,합계,2021년 11월 29일 00시,444200,3286
12936,합계,2021년 11월 30일 00시,447230,3003


In [60]:
합계_df = grouped[grouped['시도명']=='합계']
합계_df

,시도명,날짜,확진자 수,지역발생 수
12240,합계,2020년 01월 20일 00시,1,0
12241,합계,2020년 01월 21일 00시,1,0
12242,합계,2020년 01월 22일 00시,1,0
12243,합계,2020년 01월 23일 00시,1,0
12244,합계,2020년 01월 24일 00시,2,0
...,...,...,...,...
12915,합계,2021년 11월 26일 00시,432901,3882
12916,합계,2021년 11월 27일 00시,436968,4045
12917,합계,2021년 11월 28일 00시,440896,3893
12918,합계,2021년 11월 29일 00시,444200,3286


## 일일 확진자 도시별로 시트별로 저장

In [8]:
# 유일한 값 찾기
city_list = grouped['시도명'].unique().tolist()
city_list

['강원',
 '검역',
 '경기',
 '경남',
 '경북',
 '광주',
 '대구',
 '대전',
 '부산',
 '서울',
 '세종',
 '울산',
 '인천',
 '전남',
 '전북',
 '제주',
 '충남',
 '충북',
 '합계']

In [9]:
df_list = []

for city in city_list:
    df_list.append(grouped[grouped['시도명']==city])

In [10]:
df_list

[    시도명                 날짜 확진자 수 지역발생 수
 0    강원  2020년 01월 20일 00시     0      0
 1    강원  2020년 01월 21일 00시     0      0
 2    강원  2020년 01월 22일 00시     0      0
 3    강원  2020년 01월 23일 00시     0      0
 4    강원  2020년 01월 24일 00시     0      0
 ..   ..                ...   ...    ...
 676  강원  2021년 11월 27일 00시  8437     93
 677  강원  2021년 11월 28일 00시  8500     61
 678  강원  2021년 11월 29일 00시  8556     55
 679  강원  2021년 11월 30일 00시  8640     84
 680  강원  2021년 12월 01일 00시  8745    105
 
 [681 rows x 4 columns],
      시도명                 날짜 확진자 수 지역발생 수
 681   검역  2020년 01월 20일 00시     0      0
 682   검역  2020년 01월 21일 00시     0      0
 683   검역  2020년 01월 22일 00시     0      0
 684   검역  2020년 01월 23일 00시     0      0
 685   검역  2020년 01월 24일 00시     0      0
 ...   ..                ...   ...    ...
 1357  검역  2021년 11월 27일 00시  6428      0
 1358  검역  2021년 11월 28일 00시  6438      0
 1359  검역  2021년 11월 29일 00시  6441      0
 1360  검역  2021년 11월 30일 00시  6449      0
 1361  검역  2021년 12

In [61]:
import datetime as dt
from datetime import datetime, timedelta

def get_date(y, m, d):
    s = f'{y:04d}-{m:02d}-{d:02d}'
    return datetime.strptime(s, '%Y-%m-%d')

def get_week_no(y, m, d):
    target = get_date(y, m, d)
    firstday = target.replace(day = 1)
    if firstday.weekday() == 6:
        origin = firstday
    elif firstday.weekday() < 3:
        origin = firstday - timedelta(days=firstday.weekday() + 1)
    else:
        origin = firstday + timedelta(days = 6-firstday.weekday())
    return (target - origin).days // 7 + 1

In [62]:
n = dt.datetime.now()

writer = pd.ExcelWriter('국내 코로나 확진자-지역별 일일 현황_{}년 {}월 {}주차.xlsx'
                        .format(n.year, n.month, get_week_no(n.year, n.month, n.day)), 
                        engine = 'xlsxwriter')

In [71]:
for i, df in enumerate(df_list):
    print(city_list[i])
    print(df)
    # df.to_excel(writer, index = False, sheet_name = city_list[i])
    
# writer.save()

강원
    시도명                 날짜 확진자 수 지역발생 수
0    강원  2020년 01월 20일 00시     0      0
1    강원  2020년 01월 21일 00시     0      0
2    강원  2020년 01월 22일 00시     0      0
3    강원  2020년 01월 23일 00시     0      0
4    강원  2020년 01월 24일 00시     0      0
..   ..                ...   ...    ...
675  강원  2021년 11월 26일 00시  8344     53
676  강원  2021년 11월 27일 00시  8437     93
677  강원  2021년 11월 28일 00시  8500     61
678  강원  2021년 11월 29일 00시  8556     55
679  강원  2021년 11월 30일 00시  8640     84

[680 rows x 4 columns]
검역
     시도명                 날짜 확진자 수 지역발생 수
680   검역  2020년 01월 20일 00시     0      0
681   검역  2020년 01월 21일 00시     0      0
682   검역  2020년 01월 22일 00시     0      0
683   검역  2020년 01월 23일 00시     0      0
684   검역  2020년 01월 24일 00시     0      0
...   ..                ...   ...    ...
1355  검역  2021년 11월 26일 00시  6423      0
1356  검역  2021년 11월 27일 00시  6428      0
1357  검역  2021년 11월 28일 00시  6438      0
1358  검역  2021년 11월 29일 00시  6441      0
1359  검역  2021년 11월 30일 00시  6449      